# Výběr snímků z videa
mit video moc rychle
## Focus measure
[https://www.pyimagesearch.com/2015/09/07/blur-detection-with-opencv/]

Nejkvalitnější snímky z videa jsou ty s nejmenším rozmazáním. Méně rozmazaný snímek zároven zřejmě bude mít výraznější hrany než ten více rozmazaný. Hrany detekujeme pomocí konvoluce Laplaceova operátoru s vybraným snímkem. Výsledný (absolutní) průměr této matice nám bude sloužit jako nástroj k porovnávání rozmazání mezi snímky.
Tato metoda není úplně přesná, protože se scéna na snímcích konstantě mění, nicméně se nemění tak dramaticky, aby nebyla použitelná.

## Velikost okna
Předpokládáme rychlost kamery 20 cm/s a frame rate 30FPS ,tedy posun 2/3 cm/frame.
Chceme aby se dva snímky překrývali alespon ze 1/3 (při menším překryvu by počítání tr.mat mohlo být obtížné/nepřesné) a zároven se nepřerývali více než ze 1/2 (minimalizace počtu snímků).
Předpokládáme, že snímek zobrazuje vertikálně 1 m ve skutečnosti. Pak se dostáváme na hodnotu okna 37-75 framů.

Nápad : Počítání rychlosti kamery. 2 kola
1) Vyberu dostatečně ostré snímky (jen na začátku/ celé video?). Možná budu potřebovat pevné okno? 

2) Spočítám si projektivní tr. matice.

3) Pomocí velikosti posunu tr.matice a časovou vzdáleností spočítám průměrnou rychlost.

4) Přes známou rychlost spočítám přesnější okno

100 frameu uprostred-spocitam rychlost z toho
dynamicke zvetsivano/zmensovani okna podle predchoziho-lepsi

## Error threshold

Pro odhad dobré transformace použijeme následující metodu: Označíme smer. odchylku intenzity snímku jako errThresh. Tato hodnota nám bude sloužit jako prah při porovnávání s chybou transformace. Ta je spočítána jako sum(abs(I1-I2)) kde I1,I2 jsou překrývající se části snímků.
Pokud se chyba žádného snímku nevejde pod ErrThresh, je vybrán snímek s nejmenší chybou transformace v daném okně.

mean     = sum(x)/length(x)
variance = sum((x - mean(x)).^2)/(length(x) - 1);
std = sqrt(variance)


## Pipeline
Pro všechny snímky ve videu je spočítána focus measure. Všechny ostatní práce se snímky je postupně dělána ve vyhrazených oknech, což je soubor snímků vzdálený od předchozího vybraného snímku minimální a maximální vzáleností(vzdáleností je myšleno časové pořadí snímků). Pro první průchod je jednoduše vybrán snímek s největší fm a spočítán ErrThresh pomocí něj. Zbytek cyklu probíhá takto:

1) Všechny snímky okna jsou v sestupném pořadí seřazeny podle jejich focus measure.

2) Snímky jsou jeden po druhém registrovány na vybraný snímek z předchozího okna, dokun není nalezen snímek s chybou transformace menší než ErrThresh. Pokud ani jeden není menší, je vybrán snímek s nejmenší chybout tr.

3) Tento snímek je přidán do konečného výběru snímků 

# Registrace panoramatu
[https://www.mathworks.com/help/vision/ug/feature-based-panoramic-image-stitching.html]

## computeTforms
Transfromační matice jsou počítány následující způsobem:
Snímek je převeden do černobílý (1 vrstva místo 3 u RGB) a jsou naněm detekovány body pomocí SURF detektoru. Z těchto bodů jsou vybrány příznaky, které jsou následně spárovány s příznaky z předchozího snímku.
Tyto 2 seznamy jsou poté použity funkcí estimateGeometricTransform, která spočítá odpovídající tr. matici.
Pro potřeby panoramatu je použita affiní transformace, protože u projektivní dochází při součinu matic ke velké chybě.

Pokud chceme, aby referenčním snímkem v panoramatu byl jiný než první, je potřeba je přepočítat pomocí 
tforms(j).T = tforms(j).T * Tinv.T; kde Tinv je inverze ref. snímku.

## computeLimits
Je třeba znát rozměry výsledného panoramatu, čehož docílíme transformací 4 bodů okrajů u všech snímku a následným výběrem těch nejzaších. V tomto kroce je dobré ověřit velikost pro výsledné panorama, pokud je příliš velká, evidentně někde nastala chyba.

## createPanorama
Díky znalosti rozměrů panoramatu si můžeme vytvořit konečný souřadný systém, do kterého nejdříve vložíme masku (opět 4 kraje původního snímku) a následně do místa masky vložíme transformovaný snímek. (Masku pak můžeme využít na zobrazení vizualizaci poskládání všech snímků)

## Omezení 
Jak bylo výše uvedeno, u dlouhých panoramat s jakými pracujeme my, je třeba použít affiní, či podobnostní transformaci, nebot při součinu projektivních transformací dochází ke stále větší a větší chybě, a při více než 5-7 snímcích ani nelze žádné panorama vytvořit.

# Repeatability rate

[https://hal.inria.fr/inria-00548302/document]

Repeatability rate definujeme jako počet stejných bodů vyskytujících se na obou snímcích vzhledem k celkovém počtu detekovaných bodů. Z detekovaných bodů vybíráme pouze ty, které jsou součástí scény vyskytujících se na obou snímcích.

![repeat](aaa.png)

Je důležité vzít v úvahu nepřesnost detekce. Korespondující bod se nebude nacházet přesně daném místě určeném maticí, ale v jeho $\epsilon$-okolí. $\epsilon$-reapeatabilitu definujeme jako R<sub>i</sub>($\epsilon$) = {(x<sub>i</sub>,x<sub>j</sub>)|dist(H<sub>ij</sub>x<sub>i</sub>, x<sub>j</sub>) < $\epsilon$}

Počet detekovaných bodů ve společné části snímku nebude stejný, proto vybereme menší z nich, abychom zajistili existenci bodu z $\epsilon$-reapeatability. Repeatability rate je tedy definováno následovně:
 r<sub>i</sub>($\epsilon$)=abs(R<sub>i</sub>($\epsilon$))/min(n<sub>i</sub>,<sub>j</sub>)

# Vliv rozmazání na registraci

Pro sledování vlivu rozmazání na kvalitu registrace použijeme dvojice sousedících snímků, které uměle rozmažeme aplikací motion-blur filtru náhodné orientace v rozsahu <0,190> a pevně zvolené délky v rozsahu <1,50>, kdy je postupně přidávana po inkrementech 5(pixelů?). Z jedné dvojice tak získáme 11 (12 s originálními) párů rozmazaných snímků. U nich pak najdeme příznaky, namatchujeme je a odhadneme homografní matici. Pro E v Eps-repeatabilite dosadíme 10. Takte získáme repeatability rate.


Dále chceme znát localization error, tedy průměrnou vzdálenost bodu predikovaného tr. maticí od místa, kde se bod skutečně nachází. To se dá vyjádřit jako:
dist(i) =  norm(p2(i,:)-predicted(i,:)); mean(dist)

Takto projdeme celý daný set snímků. Tady je graf setu 1:
![title](rep.png)
Na vertikální ose je repeatability rate a jeho rozptyl, na horizontální délka filtru. Jak lze vidět z grafu, do cca 30 pixelu je kvalita registrace poměrně spolehlivá, pak ale začne rychle klesat.

# Vliv artefaktů

Sledování vlivu zmenšování datového toku kvalitu registrace.
Plánovaný postup:

1)Vygeneruju set videí se různým datovým tokem

2)Problém s výběrem snímku (od nějakého to už nezvládne vybrat snímek)

   řešení: vybírám předem dané snímky (znám čísla z originálu)
    
3)Dále postup stejný jako u bluru (detekce příznaků, matching, estimateGeometricTransform,etc...))

# Sčítání cenovek

V této části si ukážeme využití výše zmíněných metod na problému počítání cenovek. V podstatě jde o schopnost detekce stejného objektu na více snímcích a následném rozdělění tak, aby každý snímek měl "své" cenovky. Pro toto je využití vlastností transfromačních matic ideální.

Pro samotnou detekci cenovek byla využita typ síte YOLOv5[https://github.com/ultralytics/yolov5] natrénována na cenovky od firmy DataSentics.
Ta vyniká především pro svou rychlost i malou velikost, čímž je vhodná pro využití v mobilních zařízeních.
Vstupem je snímek a výstupem list souřadnic bounding boxů.

Snímky jsou získany z videa postupem výše uvedeném. Mezi těmito snímky je spočítany matice projektivní transformace a je také (odděleně) vygenerováno panorama pomocí affiní transformace. Toto panoram slouží k ručnímu
sčítání cenovek.

V každém cyklu jsou vzaty 3 po sobě jsoucí snímky, v nichž jsou následně detekovány cenovky. První a třetí bboxy jsou převedeny transformací do souřadného systému druhého snímku. Poté jsou přes Python knihovnu Shapely[] souřadnice bboxů převedeny do polygonů pro snadnější manipulaci. 

Polygony prvního a druhého snímku jsou dávány přes sebe a pokud je nalezen překryv, polygon je odstraněn z list ve 3. a zůstává pouze ve 1.
Následně jsou podobným způsoben kontrolovány polygony 1.&2. a 3.&2. Při nalezení překryvu ale o tom, ze kterého listu bude polygon smazán rozhoduje velikost obsahu cenovky.

Po prvním cyklu neprochází již detekované snímky sítí znovu, místo toho jsou brány seznamy z předchozích 2 cyklů.
Díky tomu nedochází k duplikaci cenovek na různých snímcích. Mohlo by v nějakých případech dojít k duplikaci, pokud by se objekt objevil na 4 a více snímcích po sobě, nicméně k tomu v podstatě nedochází, vzhledem k tomu že snímky jsou vybrány z videa, tak, aby neměly tak velký překryv. Nehledě na to, že při součinu projektivních transformačních matic se rychle kumuluje chyba po několika cyklech by se bboxy přestaly překrývat.


## Omezení
Sít má problém s cenovkami velmi blízko u sebe a najde neexistující cenovky. Také má problém správně detekovat cenovky vyloženě přes sebe a neobvyklých formátů(elektronické, moc dlouhé)

## Výsledky
ručně/program

set30-188/190

set34-102/96

mít 5 setů - vizualizace pomocí panoramatu s cenovkami přes